# Bronze Layer Ingestion for Stock Quotes

In [1]:
# This notebook grabs the latest stock quote data from the Finnhub API
# and drops it into the Bronze layer of our Lakehouse in raw JSON format.
# We’re using Hive-style partitioning here—it helps keep things organized and speeds up downstream processing.

import requests
import json
from datetime import datetime
from notebookutils import mssparkutils

# --- Configuration ---
# Just a heads-up: in production, we should manage secrets like API keys securely—
# something like Azure Key Vault works great for that.

FINNHUB_API_KEY = 'd2gdc2hr01qq1lhui8e0d2gdc2hr01qq1lhui8eg' 
STOCK_SYMBOL = 'AAPL'
BASE_URL = 'https://finnhub.io/api/v1/quote'

try:
# Step 1: Let’s fetch the latest stock quote data from the Finnhub API.
# This gives us up-to-date market info to kick off our data pipeline.
    url = f"{BASE_URL}?symbol={STOCK_SYMBOL}&token={FINNHUB_API_KEY}"
    response = requests.get(url)
    response.raise_for_status()  # Ensures we stop on HTTP errors (4xx or 5xx).
    
    stock_data = response.json()
    print(f"Successfully fetched data for {STOCK_SYMBOL}: {stock_data}")

# Step 2: Let’s define the path to our Bronze layer using Hive-style partitioning.
# This setup helps organize the data by key attributes—making it easier and faster to process later on.
    now = datetime.now()
    relative_directory_path = f"Files/bronze/trades/stock_symbol={STOCK_SYMBOL}/year={now.strftime('%Y')}/month={now.strftime('%m')}/day={now.strftime('%d')}"
    file_name = f"{now.strftime('%Y%m%d_%H%M%S')}.json"
    full_relative_path = f"{relative_directory_path}/{file_name}"
    
    print(f"Target path: {full_relative_path}")

# Step 3: Now let’s save the raw JSON payload into the Bronze layer.
# This gives us a reliable snapshot of the incoming data—exactly as it arrived from the source.
    json_data_string = json.dumps(stock_data, indent=4)
    
# Before we write the file, let’s make sure the target directory actually exists.
# It’s a simple check that saves us from annoying write errors down the line.
    mssparkutils.fs.mkdirs(relative_directory_path)
    mssparkutils.fs.put(full_relative_path, json_data_string, True)
    
    print(f"SUCCESS: Raw JSON for {STOCK_SYMBOL} landed in Bronze layer.")

except requests.exceptions.RequestException as e:
    print(f"API Call Failed: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

StatementMeta(, 87562db5-4053-422b-9d88-5df80c7888c4, 3, Finished, Available, Finished)

Successfully fetched data for AAPL: {'c': 227.3597, 'd': 2.4597, 'dp': 1.0937, 'h': 228.72, 'l': 225.35, 'o': 225.35, 'pc': 224.9, 't': 1755878994}
Target path: Files/bronze/trades/stock_symbol=AAPL/year=2025/month=08/day=22/20250822_161034.json
SUCCESS: Raw JSON for AAPL landed in Bronze layer.
